In [ ]:
import time
from decimal import Decimal

import pandas as pd

from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.backtest.models import FillModel
from nautilus_trader.backtest.modules import FXRolloverInterestConfig
from nautilus_trader.backtest.modules import FXRolloverInterestModule
from nautilus_trader.config import RiskEngineConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.examples.strategies.ema_cross import EMACross
from nautilus_trader.examples.strategies.ema_cross import EMACrossConfig
from nautilus_trader.model.currencies import USD, JPY
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import TestDataProvider
from nautilus_trader.test_kit.providers import TestInstrumentProvider

In [ ]:
# Configure backtest engine
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
    logging=LoggingConfig(log_level="ERROR"),
    risk_engine=RiskEngineConfig(
        bypass=True,  # Example of bypassing pre-trade risk checks for backtests
    ),
)

# Build backtest engine
engine = BacktestEngine(config=config)

In [ ]:
# Optional plug in module to simulate rollover interest,
# the data is coming from packaged test data.
provider = TestDataProvider()
interest_rate_data = provider.read_csv("short-term-interest.csv")
config = FXRolloverInterestConfig(interest_rate_data)
fx_rollover_interest = FXRolloverInterestModule(config=config)

# Create a fill model (optional)
fill_model = FillModel(
    prob_fill_on_limit=0.2,
    prob_fill_on_stop=0.95,
    prob_slippage=0.5,
    random_seed=42,
)

# Add a trading venue (multiple venues possible)
SIM = Venue("SIM")
engine.add_venue(
    venue=SIM,
    oms_type=OmsType.HEDGING,  # Venue will generate position IDs
    account_type=AccountType.MARGIN,
    base_currency=None,  # Standard single-currency account
    starting_balances=[Money(1_000_000, USD), Money(10_000_000, JPY)],  # Single-currency or multi-currency accounts
    fill_model=fill_model,
    modules=[fx_rollover_interest],
)

In [ ]:
# Add instruments
USDJPY_SIM = TestInstrumentProvider.default_fx_ccy("USD/JPY", SIM)
engine.add_instrument(USDJPY_SIM)

# Add data
wrangler = QuoteTickDataWrangler(instrument=USDJPY_SIM)
ticks = wrangler.process_bar_data(
    bid_data=provider.read_csv_bars("fxcm-usdjpy-m1-bid-2013.csv"),
    ask_data=provider.read_csv_bars("fxcm-usdjpy-m1-ask-2013.csv"),
)
engine.add_data(ticks)

In [ ]:
# Configure your strategy
config = EMACrossConfig(
    instrument_id=str(USDJPY_SIM.id),
    bar_type="USD/JPY.SIM-5-MINUTE-BID-INTERNAL",
    fast_ema_period=10,
    slow_ema_period=20,
    trade_size=Decimal(1_000_000),
)

# Instantiate and add your strategy
strategy = EMACross(config=config)
engine.add_strategy(strategy=strategy)

In [ ]:
# Run backtest
engine.run()

In [ ]:
engine.trader.generate_account_report(SIM)

In [ ]:
engine.trader.generate_order_fills_report()

In [ ]:
engine.trader.generate_positions_report()

In [ ]:
# For repeated backtest runs make sure to reset the engine
engine.reset()

In [ ]:
# Good practice to dispose of the object when done
engine.dispose()